In [1]:
import pandas
pandas.options.mode.chained_assignment = None
import sys
import numpy as np
from globsML.utils.preprocessing import rescale_data, select_galaxies, create_data_dict, replace_NaN
from globsML.models.skmodels import StandardClassifier

data_path = '../../../data/ACS_sources_original.csv'
data = pandas.read_csv(data_path)
test_galaxies = set(data[data['cluster']=='Virgo']['galaxy'].unique())

In [2]:
df_train, dfs_test = select_galaxies(data, test_galaxies = test_galaxies)
df_train, dfs_test = replace_NaN(df_train, dfs_test)
df = create_data_dict(data, df_train, dfs_test)
df,scaler = rescale_data(df)

input_dim = len(df['train']['inputs'][0])

Number of sources in training split after dropping rows with NaN as CI/m/color: 21232
535 sources have been dropped.
NaN values in testing data have been replaced with the correspnding median value observed in the training split


100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 3457.20it/s]

eccentricity will not be transformed. Skipped.
eccentricity_z will not be transformed. Skipped.


In [11]:
model = StandardClassifier(method='tabnet', params={'n_steps': 5, 'n_d': 10, 'n_a': 10}, data=df)

Device used : cuda
Loading data...
Fitting model...
epoch 0  | loss: 0.7053  | val_0_auc: 0.78368 |  0:00:01s
epoch 1  | loss: 0.31041 | val_0_auc: 0.92938 |  0:00:02s
epoch 2  | loss: 0.2601  | val_0_auc: 0.95239 |  0:00:04s
epoch 3  | loss: 0.22368 | val_0_auc: 0.96443 |  0:00:05s
epoch 4  | loss: 0.20892 | val_0_auc: 0.97531 |  0:00:07s
epoch 5  | loss: 0.18819 | val_0_auc: 0.9786  |  0:00:08s
epoch 6  | loss: 0.1793  | val_0_auc: 0.9754  |  0:00:10s
epoch 7  | loss: 0.17219 | val_0_auc: 0.97858 |  0:00:12s
epoch 8  | loss: 0.1605  | val_0_auc: 0.9809  |  0:00:13s
epoch 9  | loss: 0.15498 | val_0_auc: 0.98198 |  0:00:16s
epoch 10 | loss: 0.15218 | val_0_auc: 0.98058 |  0:00:18s
epoch 11 | loss: 0.14576 | val_0_auc: 0.98335 |  0:00:20s
epoch 12 | loss: 0.14435 | val_0_auc: 0.98265 |  0:00:22s
epoch 13 | loss: 0.14177 | val_0_auc: 0.98335 |  0:00:23s
epoch 14 | loss: 0.14075 | val_0_auc: 0.98626 |  0:00:24s
epoch 15 | loss: 0.13894 | val_0_auc: 0.98524 |  0:00:26s
epoch 16 | loss: 0.1

In [12]:
model.val()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
model.stats_validation

,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,0.920128,0.09434,0.040053,0.948554,0.986485,288,313,30,1062


In [14]:
model.test()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dodo/git/globsML/globsML/utils/eval.py:146: RuntimeWarning: divide by zero encountered in double_scalars
  return auc(x,y)/(2*auc([0,x[-1]], [0,y[-1]])), x, y


In [15]:
model.stats_galaxies

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,VCC575,0.913043,0.382353,0.034667,0.894378,0.988580,21,23,13,398
1,VCC1695,0.863636,0.441176,0.030675,0.772999,0.989078,19,22,15,511
2,VCC1355,0.750000,0.400000,0.032468,0.813513,0.984334,15,20,10,328
3,VCC1743,0.533333,0.818182,0.042303,0.348243,0.942852,8,15,36,866
4,VCC1886,0.692308,0.608696,0.018767,0.622898,0.931584,9,13,14,759
...,...,...,...,...,...,...,...,...,...,...
93,VCC784,0.951613,0.092308,0.015038,0.969314,0.995493,59,62,6,461
94,VCC1087,0.865672,0.256410,0.043764,0.917243,0.979539,58,67,20,524
95,VCC1185,0.848485,0.377778,0.022576,0.848605,0.990161,28,33,17,786
96,VCC1146,0.938272,0.126437,0.024831,0.972649,0.995457,76,81,11,524


In [16]:
model.stats_galaxies.mean()

/tmp/ipykernel_1135/3415850949.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  model.stats_galaxies.mean()


TPR               0.855774
FDR               0.285262
FPR               0.038480
AUC(FDR,TPR)           inf
AUC(FPR,TPR)      0.975996
# found GCs     115.357143
# total GCs     126.479592
# fake GCs       18.132653
# sources       644.510204
dtype: float64

In [17]:
model.stats_all

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,ALL,0.912061,0.135835,0.035003,0.944575,0.98523,11305,12395,1777,63162
